# Реализация ядерного сглажиания

In [1]:
import numpy as np
from statsmodels.nonparametric.kernel_regression import KernelReg
from sklearn.model_selection import KFold
from scipy.optimize import minimize

def kernel_regression_silverman(x_train, y_train, x_test):
    x_train = np.asarray(x_train).reshape(-1, 1)
    h = 1.06 * np.std(x_train) * len(x_train)**(-1/5)
    kr = KernelReg(y_train, x_train, var_type='c', bw=[h])
    return kr.fit(x_test.reshape(-1, 1))[0]

def kernel_regression_cv(x_train, y_train, x_test):
    x_train = np.asarray(x_train).reshape(-1, 1)
    x_test = np.asarray(x_test).reshape(-1, 1)
    kf = KFold(5)
    
    def cv_mse(log_h):
        h = np.exp(log_h)
        total_mse = 0
        for train_idx, val_idx in kf.split(x_train):
            kr = KernelReg(y_train[train_idx], x_train[train_idx], var_type='c', bw=[h])
            pred = kr.fit(x_train[val_idx])[0]
            total_mse += np.mean((pred - y_train[val_idx])**2)
        return total_mse / kf.n_splits
    
    h_init = np.log(1.06 * np.std(x_train) * len(x_train)**(-1/5))
    res = minimize(cv_mse, x0=h_init, method='L-BFGS-B')
    best_h = np.exp(res.x[0])
    
    kr_final = KernelReg(y_train, x_train, var_type='c', bw=[best_h])
    return kr_final.fit(x_test)[0]

# Проверка